#### NOTE: This is the pipeline of Activity recognition without any description This is the size of a typical notebook

In [1]:
import sys
%matplotlib inline

from sensiml import SensiML
from sensiml.widgets import *

dsk = SensiML()

In [2]:
dsk.project ='Activity Demo'
dsk.pipeline = "Activity Demo"

### Make a query

In [5]:
QueryWidget(dsk).create_widget()

HBox(children=(VBox(children=(Dropdown(description=u'Select Query', options=('', u'Q1', u'query_activity_demo', u'query_activity_wrist_twist', u'query_activity', u'query', u'query_activity_test'), value=''), Text(value=u'', description=u'Query Name'), Dropdown(description=u'Segmenter', options=(u'Manual',), value=u'Manual'), Dropdown(description=u'Label', options=(u'Activity', u'Subject'), value=u'Activity'), SelectMultiple(description=u'Metadtata', index=(1,), options=(u'Activity', u'Subject'), value=(u'Subject',)), SelectMultiple(description=u'Sources', index=(0, 1, 2, 3, 4, 5), options=(u'AccelerometerX', u'AccelerometerY', u'AccelerometerZ', u'GyroscopeX', u'GyroscopeY', u'GyroscopeZ'), value=(u'AccelerometerX', u'AccelerometerY', u'AccelerometerZ', u'GyroscopeX', u'GyroscopeY', u'GyroscopeZ')), Text(value=u'', description=u'Query Filter'), HBox(children=(Button(description=u'Add Query', style=ButtonStyle()), Button(description=u'Update Query', style=ButtonStyle()))))), Figure(background_style={'fill': 'white'}, fig_margin={'top': 10, 'left': 50, 'right': 0, 'bottom': 30}, layout=Layout(height=u'400px', min_width=u'125px', width=u'560px'), legend_location='bottom-right', scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))))

### Construct a pipeline

#### Generate Features

In [6]:
dsk.pipeline.reset()
dsk.pipeline.set_input_query('query_activity')

dsk.pipeline.add_transform("Magnitude", params={"input_columns": ['GyroscopeX','GyroscopeY', 'GyroscopeZ']})

dsk.pipeline.add_transform("Windowing", params={"window_size": 300, 
                                                "delta": 300,})

dsk.pipeline.add_transform("MSE Filter", params={"input_column": 'Magnitude_ST_0000', 
                                                "MSE_target": -1.0, 
                                                "MSE_threshold": 0.01})

dsk.pipeline.add_feature_generator(["Mean", 'Standard Deviation','Sum', '25th Percentile'],
                                   function_defaults = {"columns":[u'Magnitude_ST_0000']})

dsk.pipeline.add_transform('Min Max Scale')

features, s = dsk.pipeline.execute()

Executing Pipeline with Steps:

------------------------------------------------------------------------
 0.     Name: query_activity            		Type: query                    
------------------------------------------------------------------------
------------------------------------------------------------------------
 1.     Name: Magnitude                 		Type: transform                
------------------------------------------------------------------------
------------------------------------------------------------------------
 2.     Name: Windowing                 		Type: segmenter                
------------------------------------------------------------------------
------------------------------------------------------------------------
 3.     Name: MSE Filter                		Type: transform                
------------------------------------------------------------------------
------------------------------------------------------------------------
 4.     Name: g

#### Train the Model

In [9]:
dsk.pipeline.set_validation_method('Stratified K-Fold Cross-Validation', params={'number_of_folds':3,})

dsk.pipeline.set_classifier('PVP', params={"classification_mode":'RBF','distance_mode':'L1'})

dsk.pipeline.set_training_algorithm('Hierarchical Clustering with Neuron Optimization', params = {'number_of_neurons':5})


dsk.pipeline.set_tvo({'validation_seed':2})

model_results, stats = dsk.pipeline.execute()

model_results.summarize()

Executing Pipeline with Steps:

------------------------------------------------------------------------
 0.     Name: query_activity            		Type: query                    
------------------------------------------------------------------------
------------------------------------------------------------------------
 1.     Name: Magnitude                 		Type: transform                
------------------------------------------------------------------------
------------------------------------------------------------------------
 2.     Name: Windowing                 		Type: segmenter                
------------------------------------------------------------------------
------------------------------------------------------------------------
 3.     Name: MSE Filter                		Type: transform                
------------------------------------------------------------------------
------------------------------------------------------------------------
 4.     Name: g

In [10]:
model = model_results.configurations[0].models[0]
model.knowledgepack.save('ActivityKP')

Knowledgepack name updated.


### Download Model

In [11]:
DownloadWidget(dsk).create_widget()

VBox(children=(HBox(children=(Dropdown(description=u'Model Name', options={u'ActivityKP': u'724df577-b100-4ef3-a1f0-98a06ac921bd', u'ActivityKP2': u'e2613d70-fddc-4c35-94f3-9e2c37d5f73a'}, value=u'724df577-b100-4ef3-a1f0-98a06ac921bd'), Button(description=u'Refresh Models', icon=u'0', layout=Layout(align_items=u'flex-start', border=u'solid 2px', display=u'flex', flex_flow=u'column', justify_content=u'center', overflow=u'visible', width=u'25%'), style=ButtonStyle()), Button(description=u'Generate Knowledgepack', icon=u'0', layout=Layout(align_items=u'flex-start', border=u'solid 2px', display=u'flex', flex_flow=u'column', justify_content=u'center', overflow=u'visible', width=u'25%'), style=ButtonStyle()))), Label(value=u'Download Settings', layout=Layout(justify_content=u'flex-start', overflow=u'visible')), HBox(children=(VBox(children=(Dropdown(description=u'HW Platform', index=1, options={'Curie Reference 4.1': 1, 'Nordic Thingy 2.1': 2, 'Amulet 4.1': 0}, value=2), Dropdown(description=u'Target OS', options=('NordicSDK',), value='NordicSDK'), Dropdown(description=u'Download', options=('Binary', 'Library'), value='Binary'))), VBox(children=(Dropdown(description=u'IMU Rate', options=(100, 50, 25), value=100), Dropdown(description=u'Debug', options=(True, False), value=True), Dropdown(description=u'Test Data', options=(None,), value=None))), SelectMultiple(description=u'Output', index=(0,), options=('ble', 'led', 'serial'), value=('ble',))))))

### Flash the Model

In [12]:
FlashWidget(dsk).create_widget()

VBox(children=(HBox(children=(interactive(children=(Dropdown(description=u'Platform', options={'Curie Reference 4.1': 1, 'Nordic Thingy 2.1': 2, 'Amulet 4.1': 0}, value=1), Output()), _dom_classes=('widget-interact',)), Label(value=u'', layout=Layout(width=u'400px')))), HBox(children=(Dropdown(description=u'Binary', layout=Layout(width=u'550px'), options=(), value=None), Button(description=u'Refresh', style=ButtonStyle()))), HBox(children=(Dropdown(description=u'Flash Method', options=(), value=None), Button(description=u'Flash', style=ButtonStyle()), Button(description=u'Generate OTA', layout=Layout(visibility=u'hidden'), style=ButtonStyle())))))